# Build Kernel Latency Predictor
Users could refer to the [Guidance](../../docs/builder/build_kernel_latency_predictor.md) for details information.

In [1]:
import warnings
from silence_tensorflow import silence_tensorflow
warnings.filterwarnings('ignore')
silence_tensorflow()

In [2]:
import tensorflow as tf

# 加载模型
with tf.io.gfile.GFile('/home/wds/zhitai/graduate/projects/nn-Meter/data/pb_models/resnet18_0.pb', 'rb') as f:
    graph_def = tf.compat.v1.GraphDef()
    graph_def.ParseFromString(f.read())

# 打印模型的层次结构
for node in graph_def.node:
    print(node.name, node.op)

input_im_0 Placeholder
conv1.conv/weight Const
conv1.conv/weight/read Identity
conv1.conv/Conv2D Conv2D
conv1.bn.batchnorm/BatchNorm/beta Const
conv1.bn.batchnorm/BatchNorm/beta/read Identity
conv1.bn.batchnorm/BatchNorm/gamma Const
conv1.bn.batchnorm/BatchNorm/gamma/read Identity
conv1.bn.batchnorm/BatchNorm/moving_mean Const
conv1.bn.batchnorm/BatchNorm/moving_mean/read Identity
conv1.bn.batchnorm/BatchNorm/moving_variance Const
conv1.bn.batchnorm/BatchNorm/moving_variance/read Identity
conv1.bn.batchnorm/BatchNorm/FusedBatchNormV3 FusedBatchNorm
conv1.relu.relu/Relu Relu
conv1.maxpool/maxpool MaxPool
layer3.1.conv/weight Const
layer3.1.conv/weight/read Identity
layer3.1.conv/Conv2D Conv2D
layer3.1.batchnorm/BatchNorm/beta Const
layer3.1.batchnorm/BatchNorm/beta/read Identity
layer3.1.batchnorm/BatchNorm/gamma Const
layer3.1.batchnorm/BatchNorm/gamma/read Identity
layer3.1.batchnorm/BatchNorm/moving_mean Const
layer3.1.batchnorm/BatchNorm/moving_mean/read Identity
layer3.1.batchnorm/

In [2]:
# initialize builder config with workspace
from nn_meter.builder import builder_config
workspace = "./workspace/"
builder_config.init(workspace) 

FileNotFoundError: config file in ./workspace/ not found, please create the workspace first.Docs: https://github.com/microsoft/nn-Meter/blob/main/docs/builder/overview.md#-create-workspace-

In [3]:
# define experiment parameters
kernel_type = "conv-bn-relu"
backend = "debug_backend"
mark = "test"
error_threshold = 0.1

nn-Meter provides APIs from different level to build the latency predictor. The finest grain of APIs initialize the predictor by random sampling, and apply adaptive sampling based on previous data. Here is an example:

In [4]:
from nn_meter.builder import build_initial_predictor_by_data, build_adaptive_predictor_by_data

# initialize kernel latency predictor
predictor, data = build_initial_predictor_by_data(
    kernel_type, backend, init_sample_num=10, mark=mark
)

(nn-Meter) building kernel for conv-bn-relu...
(nn-Meter) Generate 9 kernels and save info to ./workspace/predictor_build/results/conv-bn-relu_prior_test.json Failed information are saved in ./workspace/predictor_build/results/generate_error.log (if any).
(nn-Meter) Generate 9 kernels with kernels model saved in ./workspace/predictor_build/kernels.
(nn-Meter) Profiling ...
(nn-Meter) All 0 models profiling complete. Save all success profiled results to ./workspace/predictor_build/results/profiled_conv-bn-relu.json Failed information are saved in ./workspace/predictor_build/results/profile_error.log (if any)


ValueError: Didn't find any data of conv-bn-relu for predictor training. There maybe some error about model profiling. Please check the profiling error in `<workspace>/predictor_build/results/profile_error.log`

In [6]:
# use the data for init predictor to build adaptive sampler predictor
# this method could be called iteratively to conduct more iterations of adaptive sampling
predictor, data = build_adaptive_predictor_by_data(
    kernel_type, data, backend, finegrained_sample_num=5
)

NameError: name 'data' is not defined

Another method `build_predictor_for_kernel` is provided to integrate initial sampling and adaptive sampling. In this method, both `init_sample_num` and `finegrained_sample_num` should be assigned, and the `iteration` indicates the iteration of adaptive sampling. Initial sampling is regarded as iteration 1, thus `iteration == 2` means one iteration for adaptive sampling.

In [9]:
from nn_meter.builder import build_predictor_for_kernel
predictor, data = build_predictor_for_kernel(
    kernel_type, backend, init_sample_num=10, finegrained_sample_num=5, iteration=2
)

RuntimeError: CPU dispatcher tracer already initlized

ImportError: numpy._core.multiarray failed to import

nn-Meter also provided a end-to-end method for users to build a series of general latency predictors, named `nn_meter.builder.build_latency_predictor`. This method will build predictors for all kernels in `<workspace-path>/configs/predictorbuild_config.yaml` according to their corresponding parameters. The parameters includes `INIT_SAMPLE_NUM`, `FINEGRAINED_SAMPLE_NUM`, `ITERATION`, and `ERROR_THRESHOLD`. Here is an example:

In [10]:
# build latency predictor for kernel
from nn_meter.builder import build_latency_predictor
build_latency_predictor(backend="debug_backend")


RuntimeError: CPU dispatcher tracer already initlized

ImportError: numpy._core.multiarray failed to import